In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose
from keras.constraints import max_norm

/home/sidathm/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sidathm/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sidathm/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sidathm/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
mnist = datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
noise_factor = 0.25
train_images_noisy = train_images + noise_factor*np.random.normal(loc=0.0, scale=1.0, size=train_images.shape)
test_images_noisy = test_images + noise_factor*np.random.normal(loc=0.0, scale=1.0, size=test_images.shape)
train_images_noisy = np.clip(train_images_noisy, 0., 1.)
test_images_noisy = np.clip(test_images_noisy, 0., 1.)

In [4]:
train_images = train_images.reshape(60000,28,28,1)
test_images = test_images.reshape(10000,28,28,1)
train_images_noisy = train_images_noisy.reshape(60000,28,28,1)
test_images_noisy = test_images_noisy.reshape(10000,28,28,1)

In [5]:
max_norm_value = 2.0
# auto encoder to remove noise
auto_encoder = Sequential()
auto_encoder.add(Conv2D(64, kernel_size=(3, 3), kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform', input_shape=(28,28,1)))
auto_encoder.add(Conv2D(32, kernel_size=(3, 3), kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform'))
auto_encoder.add(Conv2DTranspose(32, kernel_size=(3,3), kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform'))
auto_encoder.add(Conv2DTranspose(64, kernel_size=(3,3), kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform'))
auto_encoder.add(Conv2D(1, kernel_size=(3, 3), kernel_constraint=max_norm(max_norm_value), activation='sigmoid', padding='same'))

W0720 00:11:07.730895 140483644692288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0720 00:11:07.754076 140483644692288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0720 00:11:07.759595 140483644692288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
train_images_noisy = train_images_noisy / 255.0
test_images_noisy = test_images_noisy / 255.0
train_images = train_images / 255.0
test_images = test_images / 255.0

In [7]:
auto_encoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
from sklearn.model_selection import train_test_split
X_noise_train, X_noise_test, y_noise_train, y_noise_test = train_test_split(
    train_images_noisy, train_images, test_size=0.5, random_state=42, shuffle=True)
auto_encoder.fit(X_noise_train, y_noise_train,epochs=2)

W0720 00:11:08.050925 140483644692288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0720 00:11:08.072713 140483644692288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0720 00:11:08.076805 140483644692288 deprecation.py:323] From /home/sidathm/.local/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0720 00:11:08.691594 140483644692288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is dep

Epoch 1/2
30000/30000 [==============================] - 93s 3ms/step - loss: 0.1069 - acc: 0.8138: 0s - loss: 0.1070 - acc: 0.
Epoch 2/2
30000/30000 [==============================] - 95s 3ms/step - loss: 0.0801 - acc: 0.8149


In [8]:
denoised_train_images = auto_encoder.predict(train_images_noisy)
denoised_test_images = auto_encoder.predict(test_images_noisy)

In [9]:
# plt.imshow(denoised_images[0], cmap="gray")

In [10]:
model = models.Sequential()
model.add(layers.Conv2D(15, (7, 7), activation='relu', input_shape=(28,28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(30, (7, 7), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10))

W0720 00:15:18.827790 140483644692288 deprecation.py:506] From /home/sidathm/.local/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:
model.fit(denoised_train_images, train_labels, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 14s 236us/sample - loss: 0.2162 - acc: 0.9341
Epoch 2/10
60000/60000 [==============================] - 12s 206us/sample - loss: 0.0810 - acc: 0.9744
Epoch 3/10
60000/60000 [==============================] - 12s 205us/sample - loss: 0.0617 - acc: 0.9807
Epoch 4/10
60000/60000 [==============================] - 12s 203us/sample - loss: 0.0500 - acc: 0.9840
Epoch 5/10
60000/60000 [==============================] - 12s 202us/sample - loss: 0.0410 - acc: 0.9870
Epoch 6/10
60000/60000 [==============================] - 12s 208us/sample - loss: 0.0360 - acc: 0.9886
Epoch 7/10
60000/60000 [==============================] - 13s 217us/sample - loss: 0.0309 - acc: 0.9899
Epoch 8/10
60000/60000 [==============================] - 12s 193us/sample - loss: 0.0255 - acc: 0.9923
Epoch 9/10
60000/60000 [==============================] - 11s 185us/sample - loss: 0.0237 - acc: 0.9923
Epoch 10/10
60000/60000 [==============================] - 12s 2

In [13]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)

print('\nTest accuracy:', test_acc)

10000/10000 [==============================] - 1s 67us/sample - loss: 0.0378 - acc: 0.9885

Test accuracy: 0.9885
